<a href="https://colab.research.google.com/github/jtviegas-sandbox/datascience/blob/main/notebooks/findata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import importlib 
import os
COLAB_MODE = importlib.find_loader('google') is not None
print(f"colab mode? {COLAB_MODE}")    

In [ ]:
import json


if COLAB_MODE:
    !pip install fredapi pandas_datareader ipykernel
    from google.colab import userdata
    FRED_KEY=userdata.get('FRED_API_KEY')
else: 
    # use requirements.txt
    with open('../.secrets.json') as f:
        secrets = json.load(f)
        FRED_KEY = secrets["FRED_KEY"]

print (FRED_KEY)


In [ ]:
from fredapi import Fred
import pandas_datareader as pdr

In [ ]:
fred = Fred(api_key=FRED_KEY)

## cpi

In [ ]:
cpi_us = fred.get_series("CPALTT01USM657N").rename(columns={"CPALTT01USM657N": "CPI"})
cpi_us